In [30]:
from datetime import datetime
class App_Logger:
    def __init__(self,file="logfile.csv"):
        self.f_name =file
    def log(self,log_type,log_msg):
        now = datetime.now()
        current_time = now.strftime("%d-%m-%Y %H:%M:%S")
        f= open(self.f_name, "a+")
        f.write(current_time+","+log_type+","+log_msg+"\n")
        f.close()


In [44]:
import mysql.connector as connection
class sql:
    def __init__(self,host,user,passwd,db=""):
        self.host=host
        self.user=user
        self.passwd=passwd
        self.db=db
        self.logger=App_Logger("logfile.txt")
        self.logger.log("info", "SQL object created")
    def conn(self):
        try:
            if self.db=="":
                return connection.connect(host=self.host,user=self.user,passwd=self.passwd)
            else:
                return connection.connect(host=self.host,user=self.user, database=self.db,passwd=self.passwd)
        except Exception as e:
            self.logger.log("error", f"connection error : {str(e)}")
            print(str(e))
            
    def db_list(self):
        try:
            conn=self.conn()
            cursor = conn.cursor() #create a cursor to execute queries
            q="SHOW DATABASES"
            cursor.execute(q)
            print(cursor.fetchall())
            conn.close()
            self.logger.log("info", "DB list displayed")
        except Exception as e:
            conn.close()
            print(str(e))
            self.logger.log("error", f"db list error : {str(e)}")
    
    def create_db(self,db_name):
        try:
            conn=self.conn()
            cursor = conn.cursor()
            cursor.execute(f"create database {db_name}")
            self.db=db_name
            conn.close()
            self.logger.log("info", f"{db_name} DB created")
        except Exception as e:
            conn.close()
            print(str(e))
            self.logger.log("error", f"db not created error : {str(e)}")
            
    def create_table(self,table_name,columns):
        try:
            conn=self.conn()
            cursor = conn.cursor()
            cursor.execute(f"CREATE TABLE {table_name} ({columns})")
            conn.close()
            self.logger.log("info", f"{table_name} table created with columns: {columns}")
        except Exception as e:
            conn.close()
            print(str(e))
            self.logger.log("error", f"table not created error : {str(e)}")
            
            
    def insert(self,table_name,data):
        try:
            conn=self.conn()
            cursor = conn.cursor()
            #print(f"INSERT INTO {table_name} VALUES ({data})")
            cursor.execute(f"INSERT INTO {table_name} VALUES ({data})")
            conn.commit()
            conn.close()
        except Exception as e:
            conn.close()
            #print(str(e))
            self.logger.log("error", f"insert error : {str(e)}")
            
            
    def dump_file(self,f_name,t_name,columns,csv=False):
        try:
            f=open(f_name,"r")
            f.readline()
            self.create_table(t_name,columns)
            
            for line in f.readlines():
                if csv:
                    data="\'"+line[:-1].replace(",","\',\'")+"\'"
                    print(data)
                else:
                    data="\'"+line[:-1].replace(";","\',\'")+"\'"
                self.insert(t_name,data)
                
            self.logger.log("info", f"{f_name} file data dumped to {t_name} table")
            
        except Exception as e:
            print(str(e))
            self.logger.log("error", f"file dump error : {str(e)}")
            

    def select_db(self,db_name):
        self.db=db_name
        self.logger.log("info", f"{db_name} DB selected")

    def show(self,t_name):
        try:
            conn=self.conn()
            cursor = conn.cursor()
            cursor.execute(f"Select * from {t_name}")
            for result in cursor.fetchall():
                print(result)
            conn.close()
            self.logger.log("info", f"{t_name} table displayed")
        except Exception as e:
            conn.close()
            print(str(e))
            self.logger.log("error", f"table show error : {str(e)}")
            
        




In [45]:
ob=sql("localhost","root","2001")


In [46]:
ob.select_db("UCI")

In [51]:
columns="Chiral_indice_n INT(2),Chiral_indice_m INT(2),Initial_atomic_coordinate_u VARCHAR(10),Initial_atomic_coordinate_v VARCHAR(10),Initial_atomic_coordinate_w VARCHAR(10),Calculated_atomic_coordinates_u VARCHAR(10),Calculated_atomic_coordinates_v VARCHAR(10),Calculated_atomic_coordinates_w VARCHAR(10)"
  
ob.dump_file("carbon_nanotubes.csv","test4",columns)

In [52]:
ob.show("test3")